<a href="https://colab.research.google.com/github/peculab/PROGRAM/blob/main/hw6_%E8%AA%B2%E8%A1%A8%E6%9F%A5%E8%A9%A2%E8%88%87%E6%8F%90%E9%86%92%E7%B4%A0%E6%9D%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup

import google.generativeai as genai

# Google Auth & Sheets
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default

In [12]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [13]:
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
api_key = userdata.get('gemini')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.5-pro')

In [14]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1myM3mFwjKdn0aJdENwhue18m-gwCDFLoPKpMqw6t1L0/edit?usp=sharing"
WORKSHEET_NAME = "HW6"
TIMEZONE = "Asia/Taipei"

In [15]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
gsheets = gc.open_by_url(SHEET_URL)
sheets = gsheets.worksheet(WORKSHEET_NAME).get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的5筆資料
df.head()

,日期,星期,課程名稱,時間(起),時間(迄),地點,攜帶品,先讀章節,備註
0,2025-11-03,一,資料庫系統,9:10,12:00,誠101,"筆電, 充電器",教科書 Ch.6 SQL,
1,2025-11-03,一,生成式AI實務,13:10,16:00,科201,"耳機, 筆電",Lecture Notes Week 6,
2,2025-11-04,二,資料庫系統,9:10,12:00,誠101,"筆電, 充電器",教科書 Ch.6 SQL,
3,2025-11-04,二,生成式AI實務,13:10,16:00,科201,"耳機, 筆電",Lecture Notes Week 6,
4,2025-11-05,三,資料庫系統,9:10,12:00,誠101,"筆電, 充電器",教科書 Ch.6 SQL,


In [16]:
import pytz
from IPython.display import display, Markdown
import datetime as dt

# === 參數：指定要查的星期 ===
# 可填 "一","二","三","四","五","六","日"，或用英文 "Mon" ... "Sun"
TARGET_WEEKDAY = "三"  # 例：查星期三的課

# === 小工具函式 ===
TW_TZ = pytz.timezone(TIMEZONE)

week_map_tw = {"一":0,"二":1,"三":2,"四":3,"五":4,"六":5,"日":6}
week_map_en = {"Mon":0,"Tue":1,"Wed":2,"Thu":3,"Fri":4,"Sat":5,"Sun":6}

def normalize_weekday(w):
    if w in week_map_tw: return week_map_tw[w]
    if w in week_map_en: return week_map_en[w]
    raise ValueError("星期請用：一~日 或 Mon~Sun")

def ensure_datetime(s):
    try:
        return pd.to_datetime(s).date()
    except Exception:
        return pd.NaT

def week_monday(any_date):
    # 回到本週一
    return any_date - dt.timedelta(days=any_date.weekday())

def date_range_this_week(today=None):
    # 取得「本週一 ~ 本週日」日期區間（以台北時區為準）
    now = dt.datetime.now(TW_TZ).date() if today is None else today
    mon = week_monday(now)
    sun = mon + dt.timedelta(days=6)
    return mon, sun

def summarize_courses(day_df):
    if day_df.empty: return "本日無課"
    parts = []
    for _, r in day_df.iterrows():
        parts.append(f"{r['時間(起)']}-{r['時間(迄)']} {r['課程名稱']}（{r['地點']}）")
    return "；".join(parts)

# === 過濾指定星期的課程（自動只抓「本週」） ===
df["日期"] = df["日期"].apply(ensure_datetime)
df = df.dropna(subset=["日期"])

target_wd = normalize_weekday(TARGET_WEEKDAY)
this_mon, this_sun = date_range_this_week()

mask_week = (df["日期"] >= this_mon) & (df["日期"] <= this_sun)
mask_wd   = df["日期"].apply(lambda d: d.weekday()==target_wd)
today_df  = df[mask_week & mask_wd].copy().sort_values(["日期","時間(起)"])

# === 顯示查詢結果（指定星期的所有課） ===
display(Markdown(f"### 本週（{this_mon} ~ {this_sun}）星期{TARGET_WEEKDAY}課程"))
if today_df.empty:
    display(Markdown("> 本日無課"))
else:
    display(today_df[["日期","課程名稱","時間(起)","時間(迄)","地點","攜帶品","先讀章節","備註"]])

# === 產生一句「行前提醒」 ===
def make_one_line_reminder(rows):
    # ── 方案A：不接模型的簡單規則版（免 API Key）
    if rows.empty:
        return "本日無課，不需準備。"
    items = []
    readings = []
    for _, r in rows.iterrows():
        if str(r.get("攜帶品","")).strip():
            items.append(str(r["攜帶品"]).strip())
        if str(r.get("先讀章節","")).strip():
            readings.append(str(r["先讀章節"]).strip())
    items_txt = "；".join(dict.fromkeys(items)) if items else "一般上課用品"
    read_txt  = "；".join(dict.fromkeys(readings)) if readings else "無指定預習"
    return f"請攜帶：{items_txt}；預習：{read_txt}。"

one_line = make_one_line_reminder(today_df)
display(Markdown("#### 行前提醒（一句話）"))
display(Markdown(f"> {one_line}"))

# === 回寫「本週重點」分頁 ===
# 若該分頁不存在就自動建立
SHEET_WEEKLY = "本週重點"
try:
    wk = gc.open_by_url(SHEET_URL).worksheet(SHEET_WEEKLY)
except gspread.exceptions.WorksheetNotFound:
    wk = gc.open_by_url(SHEET_URL).add_worksheet(title=SHEET_WEEKLY, rows=100, cols=20)

# 組成回寫資料：可以一週寫一列
summary_text = summarize_courses(df[mask_week].sort_values(["日期","時間(起)"]))
now_str = dt.datetime.now(TW_TZ).strftime("%Y-%m-%d %H:%M")

row = {
    "週次起訖": f"{this_mon} ~ {this_sun}",
    "查詢星期": f"星期{TARGET_WEEKDAY}",
    "當日課程列表": summarize_courses(today_df),
    "行前提醒一句話": one_line,
    "更新時間": now_str
}

# 追加寫入（保留歷史）
existing = pd.DataFrame(wk.get_all_records())
outdf = pd.concat([existing, pd.DataFrame([row])], ignore_index=True) if not existing.empty else pd.DataFrame([row])

wk.clear()
set_with_dataframe(wk, outdf)

display(Markdown("✅ 已回寫到 **本週重點** 分頁（保留歷史列）。"))


### 本週（2025-10-27 ~ 2025-11-02）星期三課程

> 本日無課

#### 行前提醒（一句話）

> 本日無課，不需準備。

✅ 已回寫到 **本週重點** 分頁（保留歷史列）。

In [17]:
# === 加入 Gradio 編輯面板 ===
import gradio as gr

def save_edited_reminder(edited_text):
    """讓使用者編輯後回存到 Google Sheet"""
    from datetime import datetime
    import pytz
    tz = pytz.timezone(TIMEZONE)
    now_str = datetime.now(tz).strftime("%Y-%m-%d %H:%M")

    # 重新讀取目前的分頁資料
    existing = pd.DataFrame(wk.get_all_records())
    # 追加更新列
    row = {
        "週次起訖": f"{this_mon} ~ {this_sun}",
        "查詢星期": f"星期{TARGET_WEEKDAY}",
        "當日課程列表": summarize_courses(today_df),
        "行前提醒一句話": edited_text,
        "更新時間": now_str
    }
    outdf = pd.concat([existing, pd.DataFrame([row])], ignore_index=True)
    wk.clear()
    set_with_dataframe(wk, outdf)
    return f"✅ 已更新至 Google Sheet『本週重點』分頁，目前共 {len(outdf)} 筆。"

# Gradio 介面
with gr.Blocks(title="行前提醒編輯器") as demo:
    gr.Markdown("## ✏️ 行前提醒（可編輯並回存至 Google Sheet）")
    gr.Markdown("這裡會顯示模型或程式預設生成的行前提醒，使用者可以修改後按下『提交』回存。")
    reminder_box = gr.Textbox(label="行前提醒一句話", value=one_line, lines=3)
    save_btn = gr.Button("💾 提交並回存")
    status = gr.Markdown()

    save_btn.click(save_edited_reminder, inputs=[reminder_box], outputs=[status])

demo.launch(share=False, inline=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>